In [ ]:
#Imports
import time
import sys
import os
sys.path.append('../../')

from pyspark.ml import Pipeline, PipelineModel
from sparknlp.annotator import *
from sparknlp.base import DocumentAssembler, Finisher


In [ ]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("file:///" + os.getcwd() + "/../../../src/test/resources/sentiment.parquet"). \
        limit(1000)
data.cache()
data.count()
data.show()

In [ ]:
### Define the dataframe
document_assembler = DocumentAssembler() \
            .setInputCol("text")
        
### Transform input to appropriate schema
#assembled = document_assembler.transform(data)

In [ ]:
### Sentence detector
sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")
#sentence_data = sentence_detector.transform(checked)

In [ ]:
### Tokenizer
tokenizer = Tokenizer() \
            .setInputCols(["sentence"]) \
            .setOutputCol("token")
#tokenized = tokenizer.transform(assembled)

In [ ]:
normalizer = Normalizer() \
            .setInputCols(["token"]) \
            .setOutputCol("normal")

In [ ]:
### Spell Checker
spell_checker = NorvigSweetingApproach() \
            .setInputCols(["normal"]) \
            .setOutputCol("spell")

#checked = spell_checker.fit(tokenized).transform(tokenized)

In [ ]:
sentiment_detector = ViveknSentimentApproach() \
    .setInputCols(["spell", "sentence"]) \
    .setOutputCol("sentiment") \
    .setPruneCorpus(0) \
    .setPositiveSource("file:///" + os.getcwd() + "/../../../src/test/resources/vivekn/positive") \
    .setNegativeSource("file:///" + os.getcwd() + "/../../../src/test/resources/vivekn/negative") \


In [ ]:
pos = PerceptronApproach() \
    .setInputCols(["sentence", "spell"]) \
    .setOutputCol("pos") \
    .setCorpus("file:///" + os.getcwd() + "/../../../src/test/resources/anc-pos-corpus-small/")

In [ ]:
finisher = Finisher() \
    .setInputCols(["sentiment"]) \
    .setIncludeKeys(True)

In [ ]:
pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    normalizer,
    spell_checker,
    sentiment_detector,
    pos,
    finisher
])

start = time.time()
sentiment_data = pipeline.fit(data).transform(data)
sentiment_data.show()
end = time.time()
print("Time elapsed pipeline process: " + str(end - start))

In [ ]:
for r in sentiment_data.take(5):
    print(r)

In [ ]:
start = time.time()
pipeline.write().overwrite().save("./ps")
pipeline.fit(data).write().overwrite().save("./ms")
end = time.time()
print("Time elapsed in write pipelines: " + str(end - start))

In [ ]:
start = time.time()
p = Pipeline.read().load("./ps")
pm = PipelineModel.read().load("./ms")
end = time.time()
print("Time elapsed in read pipelines: " + str(end - start))

In [ ]:
start = time.time()
pm.transform(data).where("finished_sentiment not like '%negative%'").show()
print(pm.transform(data).count())
end = time.time()
print("Time elapsed in using loaded pipelines: " + str(end - start))